In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam


In [12]:
# Path to your dataset
train_data_dir = 'D:/AI Algorithm/cucumber'

# Use ImageDataGenerator for loading and augmenting images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Set aside 20% of data for validation
)

# Load training data (80% of data)
train_data = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',  # Since this is binary classification
    subset='training'  # Use the training portion of the split
)

# Load validation data (20% of data)
val_data = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',  # Since this is binary classification
    subset='validation'  # Use the validation portion of the split
)


Found 922 images belonging to 2 classes.
Found 230 images belonging to 2 classes.


In [13]:
# Load the ResNet50 base model, pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the layers of the base model so they aren’t updated during training
base_model.trainable = False

# Add custom layers on top of the base ResNet model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sigmoid output for binary classification
])

# Summarize the model architecture
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 4, 4, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,637,313 (93.98 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [14]:
model.compile(
    loss='binary_crossentropy',  # Binary classification loss
    optimizer=Adam(learning_rate=0.0001),  # Optimizer with a small learning rate
    metrics=['accuracy']
)


In [15]:
# Train the model using the training data and validate it using the validation data
history = model.fit(
    train_data,
    epochs=20,  # Set the number of epochs as needed
    validation_data=val_data,  # Use the validation data
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]  # Early stopping
)


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - accuracy: 0.7092 - loss: 0.6197 - val_accuracy: 0.7174 - val_loss: 0.5842
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.7525 - loss: 0.5504 - val_accuracy: 0.7174 - val_loss: 0.5829
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.7188 - loss: 0.5829 - val_accuracy: 0.7174 - val_loss: 0.5743
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.7185 - loss: 0.5777 - val_accuracy: 0.7174 - val_loss: 0.5685
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.6992 - loss: 0.5907 - val_accuracy: 0.7174 - val_loss: 0.5661
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 91s 3s/step - accuracy: 0.7130 - loss: 0.5774 - val_accuracy: 0.7174 - val_loss: 0.5604
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 92s 3s/step - accuracy: 0.7407 - loss: 0.5472 - val_accuracy: 0.7174 - val_loss: 0.5585
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 102s 4s/step - accuracy: 0.7249 - loss: 0.5605 - val_accuracy: 0.7174 - val_los

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
import numpy as np


In [21]:
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    mean_absolute_error,
    mean_squared_error,
    explained_variance_score,
    r2_score,
)
from sklearn.exceptions import UndefinedMetricWarning
import warnings

# Suppress warnings for undefined metrics
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Get true classes
true_classes = val_data.classes  # Replace with actual true labels
class_indices = val_data.class_indices
class_labels = list(class_indices.keys())  # Class names

# Get predictions (class probabilities)
predictions = model.predict(val_data, verbose=1)  # Predicted probabilities
predicted_classes = np.argmax(predictions, axis=1) if predictions.ndim > 1 else (predictions > 0.5).astype(int)

# Evaluation Metrics
# Accuracy
accuracy = accuracy_score(true_classes, predicted_classes)

# Precision, Recall, F1 Score
precision = precision_score(true_classes, predicted_classes, average='weighted', zero_division=0)
recall = recall_score(true_classes, predicted_classes, average='weighted', zero_division=0)
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# ROC-AUC Score
if len(class_labels) == 2:  # Binary classification
    auc = roc_auc_score(true_classes, predictions)  # Directly use predictions
else:  # Multiclass classification
    auc = roc_auc_score(true_classes, predictions, multi_class='ovr')

# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(true_classes, predicted_classes)

# Mean Squared Error (MSE)
mse = mean_squared_error(true_classes, predicted_classes)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((true_classes - predicted_classes) / np.clip(true_classes, a_min=1, a_max=None))) * 100

# Symmetric Mean Absolute Percentage Error (SMAPE)
smape = np.mean(2 * np.abs(true_classes - predicted_classes) /
                 (np.abs(true_classes) + np.abs(predicted_classes) + 1e-10)) * 100

# Explained Variance Score
explained_variance = explained_variance_score(true_classes, predicted_classes)

# R^2 Score
r2 = r2_score(true_classes, predicted_classes)

# Display the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"AUC: {auc:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.2f}")
print(f"Explained Variance Score: {explained_variance:.2f}")
print(f"R^2 Score: {r2:.2f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
Accuracy: 0.28
Precision: 0.08
Recall: 0.28
F1-Score: 0.12
AUC: 0.41
Confusion Matrix:
[[ 65   0]
 [165   0]]
Mean Absolute Error (MAE): 0.72
Mean Squared Error (MSE): 0.72
Root Mean Squared Error (RMSE): 0.85
Mean Absolute Percentage Error (MAPE): 71.74
Symmetric Mean Absolute Percentage Error (SMAPE): 143.48
Explained Variance Score: 0.00
R^2 Score: -2.54
